# 1. Import Libraries

In [31]:
# 1. Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.utils import to_categorical
import time
import sys

# 2. Load the dataset

In [32]:
df = pd.read_csv("solar_flare_data.csv")

# Check the first few rows
print(df.head())

  modified Zurich class largest spot size spot distribution  activity  \
0                     C                 S                 O         1   
1                     D                 S                 O         1   
2                     C                 S                 O         1   
3                     D                 S                 O         1   
4                     D                 A                 O         1   

   evolution  previous 24 hour flare activity  historically-complex  \
0          2                                1                     1   
1          3                                1                     1   
2          3                                1                     1   
3          3                                1                     1   
4          3                                1                     1   

   became complex on this pass  area  area of largest spot  common flares  \
0                            2     1                     

In [33]:
# 2. Define the columns
categorical_cols = ['modified Zurich class', 'largest spot size', 'spot distribution']
ordinal_cols = ['activity', 'evolution', 'previous 24 hour flare activity',
                'historically-complex', 'became complex on this pass', 'area', 'area of largest spot']

# 3. Preprocess the data

In [34]:
# Apply One-Hot Encoding to the categorical columns
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_cols)],
    remainder='passthrough'  # Keep other columns unchanged
)

# Apply Label Encoding to the ordinal columns
label_encoders = {col: LabelEncoder() for col in ordinal_cols}

# Separate features (X) and target variables (y)
X = df.drop(columns=['common flares', 'moderate flares', 'severe flares'])  # Features
y = df[['common flares', 'moderate flares', 'severe flares']]  # Target variables

# Label encode the ordinal columns
for col in ordinal_cols:
    X[col] = label_encoders[col].fit_transform(X[col])

# Apply the column transformer to the feature data (One-Hot Encoding for categorical columns)
X_encoded = preprocessor.fit_transform(X)

# Convert the target into a single class label
# Convert the classes into 0 (C-class), 1 (M-class), and 2 (X-class)
y_class = y.idxmax(axis=1).map({'common flares': 0, 'moderate flares': 1, 'severe flares': 2})

# 4. Split the data into training and validation sets

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y_class, test_size=0.2, random_state=42)

# 5. Model Configuration

In [36]:
DENSE1_SIZE = 16
DENSE2_SIZE = 8
DENSE3_SIZE = 4
NUM_OF_EPOCHS = 100
BATCH_SIZE = 16

# 6. Build the Model

In [37]:
model = tf.keras.Sequential()

# Flatten the input data
model.add(tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)))

# Add hidden layers with ReLU activation
model.add(tf.keras.layers.Dense(DENSE1_SIZE, activation='relu'))
model.add(tf.keras.layers.Dense(DENSE2_SIZE, activation='relu'))
model.add(tf.keras.layers.Dense(DENSE3_SIZE, activation='relu'))

# Output layer with softmax activation for multi-class classification
model.add(tf.keras.layers.Dense(3, activation='softmax'))  # 3 units for 3 classes

# 7. Compile the model

In [38]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 8. Train the model

In [39]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_OF_EPOCHS, 
                    verbose=1, validation_data=(X_val, y_val))

Epoch 1/100
70/70 [==============================] - 1s 7ms/step - loss: 0.6202 - accuracy: 0.9658 - val_loss: 0.2796 - val_accuracy: 0.9676
Epoch 2/100
70/70 [==============================] - 0s 4ms/step - loss: 0.2029 - accuracy: 0.9667 - val_loss: 0.1707 - val_accuracy: 0.9676
Epoch 3/100
70/70 [==============================] - 0s 4ms/step - loss: 0.1656 - accuracy: 0.9667 - val_loss: 0.1535 - val_accuracy: 0.9676
Epoch 4/100
70/70 [==============================] - 0s 4ms/step - loss: 0.1528 - accuracy: 0.9667 - val_loss: 0.1419 - val_accuracy: 0.9676
Epoch 5/100
70/70 [==============================] - 0s 4ms/step - loss: 0.1451 - accuracy: 0.9667 - val_loss: 0.1355 - val_accuracy: 0.9676
Epoch 6/100
70/70 [==============================] - 0s 4ms/step - loss: 0.1395 - accuracy: 0.9667 - val_loss: 0.1321 - val_accuracy: 0.9676
Epoch 7/100
70/70 [==============================] - 0s 3ms/step - loss: 0.1377 - accuracy: 0.9667 - val_loss: 0.1306 - val_accuracy: 0.9676
Epoch 8/100
7

Epoch 59/100
70/70 [==============================] - 0s 4ms/step - loss: 0.0960 - accuracy: 0.9667 - val_loss: 0.1535 - val_accuracy: 0.9676
Epoch 60/100
70/70 [==============================] - 0s 4ms/step - loss: 0.0971 - accuracy: 0.9667 - val_loss: 0.1588 - val_accuracy: 0.9676
Epoch 61/100
70/70 [==============================] - 0s 4ms/step - loss: 0.0964 - accuracy: 0.9667 - val_loss: 0.1560 - val_accuracy: 0.9676
Epoch 62/100
70/70 [==============================] - 0s 4ms/step - loss: 0.0950 - accuracy: 0.9667 - val_loss: 0.1604 - val_accuracy: 0.9676
Epoch 63/100
70/70 [==============================] - 0s 3ms/step - loss: 0.0945 - accuracy: 0.9667 - val_loss: 0.1631 - val_accuracy: 0.9676
Epoch 64/100
70/70 [==============================] - 0s 3ms/step - loss: 0.0958 - accuracy: 0.9667 - val_loss: 0.1644 - val_accuracy: 0.9676
Epoch 65/100
70/70 [==============================] - 0s 4ms/step - loss: 0.0934 - accuracy: 0.9667 - val_loss: 0.1641 - val_accuracy: 0.9676
Epoch 

# 8. Evaluate the Model

In [40]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Training Accuracy: 97.21%
Validation Accuracy: 95.32%


# 9. Model Conversion to TensorFlow Lite

In [41]:
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply optimizations (optional)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Representative dataset for quantization
def representative_dataset():
    for _ in range(100):
        yield [X_train.astype(np.float32)]  # Use training data or any relevant subset

converter.representative_dataset = representative_dataset

# Convert the model
tflite_model = converter.convert()

# Save the converted model
with open('SolarFlareModel.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model has been converted and saved as 'SolarFlareModel.tflite'.")

INFO:tensorflow:Assets written to: C:\Users\Rames\AppData\Local\Temp\tmplo26olqu\assets


INFO:tensorflow:Assets written to: C:\Users\Rames\AppData\Local\Temp\tmplo26olqu\assets


Model has been converted and saved as 'SolarFlareModel.tflite'.


# 10. Save the Keras Model

In [42]:
model.save('SolarFlareModel.h5')
print("Keras model has been saved as 'SolarFlareModel.h5'.")

Keras model has been saved as 'SolarFlareModel.h5'.


# 11. Generate the C header file containing the model data

In [43]:
# Load the trained TFLite model from file 
tflite_model_path = 'SolarFlareModel.tflite'

# Read the TFLite model as byte data
with open(tflite_model_path, 'rb') as f:
    tflite_model = f.read()

# Function to convert some hex values into an array for C programming
def hex_to_c_array(hex_data, var_name):
    c_str = ""

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += "#define " + var_name.upper() + '_H\n\n'

    c_str += "/*\n Author: DR \n"
    c_str += " CAUTION: This is an auto generated file.\n DO NOT EDIT OR MAKE ANY CHANGES TO it.\n"

    # Time stamping of this model data in the generated file
    localtime = time.asctime(time.localtime(time.time()))
    c_str += " This model data was generated on " + localtime + '\n\n'
    print("This model data was generated on:", localtime)

    # Add information about the versions of tools and packages used in generating this header file
    c_str += " Tools used:\n Python: " + str(sys.version) + "\n Numpy: " + str(np.version.version) + \
             "\n TensorFlow: " + str(tf.__version__) + "\n Keras: " + str(tf.keras.__version__) + "\n\n"
    print("Tools used: Python:", sys.version, "\n Numpy:", np.version.version, \
          "\n TensorFlow:", tf.__version__, "\n Keras: ", tf.keras.__version__, "\n\n")

    # Training details of the model
    c_str += ' Model details are:\n'
    c_str += ' NUM_OF_EPOCHS = ' + str(NUM_OF_EPOCHS) + '\n'
    c_str += ' BATCH_SIZE    = ' + str(BATCH_SIZE) + '\n'
    c_str += ' DENSE1_SIZE   = ' + str(DENSE1_SIZE) + '\n'
    c_str += ' DENSE2_SIZE   = ' + str(DENSE2_SIZE) + '\n'
    c_str += ' DENSE3_SIZE   = ' + str(DENSE3_SIZE) + '\n*/\n'
    
    # Generate 'C' constants for the number of nodes in each layer
    c_str += '\nconst int ' + 'DENSE1_SIZE' + ' = ' + str(DENSE1_SIZE) + ';\n'
    c_str += 'const int ' + 'DENSE2_SIZE' + ' = ' + str(DENSE2_SIZE) + ';\n'      
    c_str += 'const int ' + 'DENSE3_SIZE' + ' = ' + str(DENSE3_SIZE) + ';\n'

    # Add array length at the top of the file
    c_str += '\nconst unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'alignas(8) const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data):
        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n'
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n' + format(''.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

header_file_path = "solar_flare_model_esp32.h"

with open(header_file_path, 'w') as file:
    file.write(hex_to_c_array(tflite_model, "solar_flare_model_esp32"))

print(f"C header file has been generated and saved as {header_file_path}.")

This model data was generated on: Sun Nov 10 13:21:38 2024
Tools used: Python: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)] 
 Numpy: 1.20.1 
 TensorFlow: 2.3.0 
 Keras:  2.4.0 


C header file has been generated and saved as solar_flare_model_esp32.h.
